In [144]:
import json
import pandas as pd
import numpy as np

with open("proj1_data.json") as fp:
    data = json.load(fp)
    df = pd.DataFrame(data)

In [145]:
df["is_root"] = df["is_root"].astype(int)
df["text"]= [x.lower() for x in df["text"]]
df["text"]= [x.split() for x in df["text"]]

In [146]:
train = df.iloc[0:10000,:]
validation = df.iloc[10000:11000,:]
test = df.iloc[11000:12000,:]
train.head(10)

,children,controversiality,is_root,popularity_score,text
0,0,0,0,1.254698,"[its, raining, sideways]"
1,0,0,0,0.509813,"[wheel, of, time, reader, confirmed!]"
2,0,0,1,0.370827,"[the, jungle, book, of, pussy]"
3,0,0,0,-0.272843,"[i'm, just, making, this, thread, since, there..."
4,0,0,1,0.560150,"[hi, there,, looks, like, you're, wanting, to,..."
5,0,0,1,0.696554,"[when, there, is, a, line, to, exit, a, store,..."
6,0,0,1,1.050417,"[it, is, none, of, their, business.]"
7,1,0,1,0.310543,"[infinite, ammo, and, the, absurd, number, of,..."
8,1,1,0,-1.208735,"[if, you're, trying, to, argue, for, a, claim,..."
9,1,0,0,1.123700,"[oh, god, it's, all, my, cousins, from, the, s..."


In [147]:
from functools import reduce
import operator
l = reduce(operator.concat, df["text"])
    

In [177]:
def preprocess(dataset):
    
    # Feature: Does the comment contain a question mark
    qmarks = np.zeros((dataset.shape[0]))
    # Feature: Normalized comment length (number of words)
    n_words = np.zeros((dataset.shape[0]))
    # Feature: Avg number of letters per words
    letters_per_word = np.zeros((dataset.shape[0]))
    
    for i in range(dataset.shape[0]):
        txt = dataset.iloc[i]["text"]
        for w in txt:
            # question counter
            if "?" in w:
                qmarks[i] = 1
                
            # comment length
            n_words[i] += 1
            # number of letters
            letters_per_word[i] += len(w)
            
    # Get average number of letters per word
    for i in range(dataset.shape[0]):
        letters_per_word[i] = letters_per_word[i]/n_words[i]
    
    # Normalize word length (divide by average)
    n_words = n_words/np.mean(n_words)
                
    # add feature columns           
    dataset = dataset.assign(has_question=pd.Series(qmarks).values.astype(int))
    dataset = dataset.assign(n_words=pd.Series(n_words).values)
    dataset = dataset.assign(letters_per_word=pd.Series(letters_per_word).values)
    
                
    return dataset

In [178]:
validation = preprocess(validation)

In [179]:
validation.head(100)

,children,controversiality,is_root,popularity_score,text,has_question,n_words,letters_per_word
10000,0,0,1,0.843337,"[kidney, infection, and, subsequent, stones.]",0,5.0,7.000000
10001,0,0,0,0.894002,"[why, have, i, never, done, this...]",0,6.0,4.000000
10002,2,0,0,3.426052,"[nothing, like, showing, a, woman, that, you, ...",0,10.0,4.700000
10003,0,0,0,0.843749,"[appreciate, the, input;, i, only, know, micro...",0,17.0,4.823529
10004,2,0,1,0.608436,"[me., waiting, on, dinner,, and, am, drunk., d...",0,24.0,4.666667
10005,1,0,0,0.411307,"[i, like, how, you, had, to, choose, assjuice1...",0,17.0,4.588235
10006,0,0,0,0.493155,"[the, season, finale, of, castle, last, year, ...",0,20.0,4.450000
10007,0,0,0,0.723746,"[working, with, sea, turtles, in, costa, rica,...",0,18.0,4.722222
10008,0,0,1,0.028793,"[anything, wrapped, in, excessive, blister, pa...",0,7.0,6.142857
10009,2,0,1,0.713490,"[the, 3, russian, teen, serial, killers, killi...",0,25.0,4.680000
